<a href="https://colab.research.google.com/github/MarynaBN/NLP_Proj/blob/main/ProjNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/docs/transformers/v4.41.0/en/tasks/sequence_classification#text-classification

In [ ]:
! pip install datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 18.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

# Pretrained Encoder-only model for text classification

In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")

In [ ]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
imdb.pop('unsupervised')
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [ ]:
seed = 0
#Let's get a small portion from the training to create a dev set
train_and_dev = imdb["train"].train_test_split(test_size=0.05, seed=seed, shuffle=True)
print(train_and_dev)

#Let's update the dataset with the new split that includes the dev data
imdb["dev"] = train_and_dev["test"]
imdb["train"] = train_and_dev["train"]
print(imdb)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 23750
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1250
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 23750
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1250
    })
})


In [ ]:
seed = 0
#Let's reduce also the test data to 5k sample only
test_and_discard = imdb["test"].train_test_split(test_size=0.2, seed=seed, shuffle=True)
print(test_and_discard)

#Let's update the test partition
imdb["test"] = test_and_discard["test"]
print(imdb)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 23750
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1250
    })
})


https://huggingface.co/distilbert/distilbert-base-cased

https://huggingface.co/distilbert/distilbert-base-uncased

In [ ]:
from transformers import AutoTokenizer

#model_name = "distilbert/distilbert-base-cased"
model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

imdb_encoded = imdb.map(tokenize_function, batched=True, batch_size=None)

# Show the sentence, the different tokens and the corresponding numerical ids
print(imdb_encoded['train'][0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'text': 'Is it a coincidence that Orca was made two years after Jaws? Orca isn\'t exactly a "Jaws rip off" but it is obvious that it tried to profit from Jaws\'s success. First of all Orca in my opinion was a bad movie, not terrible but definitely not good, average at best.<br /><br />The plot is basically a male killer whale (orca) after seeing its mate and its unborn calf killed by a fisherman seeks revenge. I couldn\'t stand to watch this movie again. The direction of this film is poor and when compared to Jaws it looks like the director, producers, and writers were almost talentless.<br /><br />As for the acting, it was very average and believable, however the actual characters aren\'t the least bit likable. The effects were alright for its time and the footage of the killer whale looked pretty good.<br /><br />The violence is confusing, bloody, and not recommended for more sensitive people. The music is overdone and very loud, drowning out the sound effects and irritating at time

https://huggingface.co/docs/transformers/v4.41.0/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification


In [ ]:
from transformers import AutoModelForSequenceClassification

print(model_name)
num_labels = 2
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id)

distilbert/distilbert-base-uncased


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
from sklearn.metrics import accuracy_score

def get_accuracy(preds):
  preds_preds = preds.predictions[0] if isinstance(preds.predictions, tuple) else preds.predictions
  predictions = preds_preds.argmax(axis=-1)
  labels = preds.label_ids
  accuracy = accuracy_score(preds.label_ids, preds_preds.argmax(axis=-1))
  return {'accuracy': accuracy}

In [ ]:
from transformers import TrainingArguments

batch_size = 16
epochs = 1
logging_steps = len(imdb_encoded["train"]) // batch_size
model_name_output_dir = model_name.replace("/", "-")+"-finetuned-imdb"
training_args = TrainingArguments(output_dir=model_name_output_dir,
                                  num_train_epochs=epochs,
                                  learning_rate=1e-4,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=get_accuracy,
                  train_dataset=imdb_encoded["train"],
                  eval_dataset=imdb_encoded["dev"],
                  tokenizer=tokenizer)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.279100,0.199472,0.924800


TrainOutput(global_step=1485, training_loss=0.2789329979832124, metrics={'train_runtime': 1160.2973, 'train_samples_per_second': 20.469, 'train_steps_per_second': 1.28, 'total_flos': 3146100718080000.0, 'train_loss': 0.2789329979832124, 'epoch': 1.0})

In [ ]:
## Save the model
trainer.save_model()

#VALIDATION ACCURACY
## Training and validation accuracy
trainer.evaluate()

{'eval_loss': 0.19947214424610138,
 'eval_accuracy': 0.9248,
 'eval_runtime': 21.2024,
 'eval_samples_per_second': 58.956,
 'eval_steps_per_second': 3.726,
 'epoch': 1.0}

In [ ]:
#TEST ACCURACY
## Get the predictions
preds = trainer.predict(imdb_encoded['test'])
print(preds)

PredictionOutput(predictions=array([[-2.2769594,  2.415001 ],
       [-2.1689346,  2.263358 ],
       [ 2.426111 , -2.451328 ],
       ...,
       [-2.2482665,  2.3490367],
       [-2.521811 ,  2.5632021],
       [-1.692059 ,  1.8386687]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 1, 1]), metrics={'test_loss': 0.2069118171930313, 'test_accuracy': 0.92, 'test_runtime': 85.4522, 'test_samples_per_second': 58.512, 'test_steps_per_second': 3.663})


In [ ]:
print(preds.metrics)

{'test_loss': 0.2069118171930313, 'test_accuracy': 0.92, 'test_runtime': 85.4522, 'test_samples_per_second': 58.512, 'test_steps_per_second': 3.663}


In [ ]:
pred_class = preds.predictions.argmax(axis=-1)
print(pred_class)
label = preds.label_ids
print(label)

[1 1 0 ... 1 1 1]
[1 1 0 ... 1 1 1]


**Test on new sentences using pipeline**

https://huggingface.co/docs/transformers/en/pipeline_tutorial


In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)
classifier('This is not my idea of fun')

[{'label': 'NEGATIVE', 'score': 0.9914997220039368}]

In [ ]:
classifier('This was beyond incredible')

[{'label': 'POSITIVE', 'score': 0.9931066632270813}]

**Linear Probing**

In [ ]:
from transformers import AutoModelForSequenceClassification

print(model_name)
num_labels = 2
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id)

distilbert/distilbert-base-uncased


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False
    else:  # classifier layer
        print(name)

pre_classifier.weight
pre_classifier.bias
classifier.weight
classifier.bias


In [ ]:
from transformers import Trainer

trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=get_accuracy,
                  train_dataset=imdb_encoded["train"],
                  eval_dataset=imdb_encoded["dev"],
                  tokenizer=tokenizer)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.418600,0.338820,0.860000


TrainOutput(global_step=1485, training_loss=0.41860993227573357, metrics={'train_runtime': 463.0911, 'train_samples_per_second': 51.286, 'train_steps_per_second': 3.207, 'total_flos': 3146100718080000.0, 'train_loss': 0.41860993227573357, 'epoch': 1.0})

In [ ]:
preds_probing = trainer.predict(imdb_encoded['test'])
print(preds_probing)

PredictionOutput(predictions=array([[-1.7473928 ,  1.8547513 ],
       [-1.7256285 ,  1.8570806 ],
       [ 1.3129106 , -1.1495267 ],
       ...,
       [-0.96515304,  1.0259078 ],
       [-3.2286963 ,  3.370456  ],
       [-1.0652593 ,  1.2378876 ]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 1, 1]), metrics={'test_loss': 0.3579508364200592, 'test_accuracy': 0.8406, 'test_runtime': 84.8357, 'test_samples_per_second': 58.937, 'test_steps_per_second': 3.689})


In [ ]:
preds_probing.metrics

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)
classifier('This is not my idea of fun')

[{'label': 'NEGATIVE', 'score': 0.6607969999313354}]

In [ ]:
classifier('This was beyond incredible')

[{'label': 'POSITIVE', 'score': 0.8769249320030212}]

# Pretrained Sequence2Sequence model for text classification

https://huggingface.co/google/t5-v1_1-small

In [ ]:
from transformers import AutoModelForSequenceClassification
model_name = 'google/t5-v1_1-small'
num_labels = 2
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

In [ ]:
model

T5ForSequenceClassification(
  (transformer): T5Model(
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (wi_1): Linear

In [ ]:
for name, param in model.named_parameters():
    if 'classification' not in name:
        param.requires_grad = False
    else:  # classification head
      print(name)

classification_head.dense.weight
classification_head.dense.bias
classification_head.out_proj.weight
classification_head.out_proj.bias


In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")

imdb.pop('unsupervised')

seed = 0
#Let's get a small portion from the training to create a dev set
train_and_dev = imdb["train"].train_test_split(test_size=0.05, seed=seed, shuffle=True)
print(train_and_dev)

#Let's update the dataset with the new split that includes the dev data
imdb["dev"] = train_and_dev["test"]
imdb["train"] = train_and_dev["train"]
print(imdb)


seed = 0
#Let's reduce also the test data to 5k sample only
test_and_discard = imdb["test"].train_test_split(test_size=0.2, seed=seed, shuffle=True)
print(test_and_discard)

#Let's update the test partition
imdb["test"] = test_and_discard["test"]
print(imdb)

from transformers import AutoTokenizer

print(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

imdb_encoded = imdb.map(tokenize_function, batched=True, batch_size=None)

# Show the sentence, the different tokens and the corresponding numerical ids
print(imdb_encoded['train'][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 23750
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1250
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 23750
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1250
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 23750
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1250
    })
})
google/t5-v1_1-small


tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Map:   0%|          | 0/23750 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

{'text': 'Is it a coincidence that Orca was made two years after Jaws? Orca isn\'t exactly a "Jaws rip off" but it is obvious that it tried to profit from Jaws\'s success. First of all Orca in my opinion was a bad movie, not terrible but definitely not good, average at best.<br /><br />The plot is basically a male killer whale (orca) after seeing its mate and its unborn calf killed by a fisherman seeks revenge. I couldn\'t stand to watch this movie again. The direction of this film is poor and when compared to Jaws it looks like the director, producers, and writers were almost talentless.<br /><br />As for the acting, it was very average and believable, however the actual characters aren\'t the least bit likable. The effects were alright for its time and the footage of the killer whale looked pretty good.<br /><br />The violence is confusing, bloody, and not recommended for more sensitive people. The music is overdone and very loud, drowning out the sound effects and irritating at time

In [ ]:
from transformers import TrainingArguments

epochs=1
batch_size = 8
logging_steps = len(imdb_encoded["train"]) // batch_size
model_name_output_dir = model_name.replace("/", "-")+"-finetuned-imdb"
training_args = TrainingArguments(output_dir=model_name_output_dir,
                                  num_train_epochs=epochs,
                                  learning_rate=1e-4,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )

In [ ]:
from sklearn.metrics import accuracy_score

def get_accuracy(preds):
  preds_preds = preds.predictions[0] if isinstance(preds.predictions, tuple) else preds.predictions
  predictions = preds_preds.argmax(axis=-1)
  labels = preds.label_ids
  accuracy = accuracy_score(preds.label_ids, preds_preds.argmax(axis=-1))
  return {'accuracy': accuracy}

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=get_accuracy,
                  train_dataset=imdb_encoded["train"],
                  eval_dataset=imdb_encoded["dev"],
                  tokenizer=tokenizer)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.700200,0.687409,0.546400


TrainOutput(global_step=2969, training_loss=0.7001148862001991, metrics={'train_runtime': 794.2963, 'train_samples_per_second': 29.901, 'train_steps_per_second': 3.738, 'total_flos': 3233979578880000.0, 'train_loss': 0.7001148862001991, 'epoch': 1.0})

In [ ]:
preds = trainer.predict(imdb_encoded['test'])
print(preds)

PredictionOutput(predictions=(array([[-0.19279699, -0.13393107],
       [-0.5810182 , -0.32953683],
       [-0.4645727 , -0.36887977],
       ...,
       [-0.501358  , -0.33752948],
       [-0.48772866, -0.39421588],
       [-0.19209637, -0.16482446]], dtype=float32), array([[[ 3.24322671e-01, -3.71217430e-01,  3.74088645e-01, ...,
         -1.24848284e-01, -4.45104316e-02, -3.90340030e-01],
        [ 1.22476518e-01, -2.89639737e-02,  3.60227019e-01, ...,
         -3.57966006e-01, -2.29919553e-01,  5.45998616e-03],
        [-7.69819021e-01,  6.31304458e-02,  6.39434904e-02, ...,
          1.23783730e-01,  3.56854349e-01, -2.17356905e-01],
        ...,
        [ 4.21546131e-01, -4.74665374e-01,  5.19849837e-01, ...,
          2.52646487e-02,  6.12153292e-01, -8.89952719e-01],
        [ 4.21546131e-01, -4.74665374e-01,  5.19849837e-01, ...,
          2.52646487e-02,  6.12153292e-01, -8.89952719e-01],
        [ 4.21546131e-01, -4.74665374e-01,  5.19849837e-01, ...,
          2.52646487e-0

In [ ]:
preds.metrics

**Fine-tuning**

In [ ]:
for name, param in model.named_parameters():
    if 'classification' not in name:
        param.requires_grad = True ####
    else:  # classifier layer
      print(name)

classification_head.dense.weight
classification_head.dense.bias
classification_head.out_proj.weight
classification_head.out_proj.bias


In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

epochs=1
batch_size = 8
logging_steps = len(imdb_encoded["train"]) // batch_size
model_name_output_dir = model_name.replace("/", "-")+"-finetuned-imdb"
training_args = TrainingArguments(output_dir=model_name_output_dir,
                                  num_train_epochs=epochs,
                                  learning_rate=1e-4,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )

trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=get_accuracy,
                  train_dataset=imdb_encoded["train"],
                  eval_dataset=imdb_encoded["dev"],
                  tokenizer=tokenizer)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.705600,0.684849,0.564000


TrainOutput(global_step=1485, training_loss=0.7055929135794592, metrics={'train_runtime': 1765.2285, 'train_samples_per_second': 13.454, 'train_steps_per_second': 0.841, 'total_flos': 3233979578880000.0, 'train_loss': 0.7055929135794592, 'epoch': 1.0})

In [ ]:
preds_probing = trainer.predict(imdb_encoded['test'])
print(preds_probing)

Example of single-label classification and Example of multi-label classification available at the following link:

https://huggingface.co/docs/transformers/v4.41.0/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification